In [1]:
from macronet.flowfunc import FuncModel as fn
import MoNet.monet as mn
import importlib
importlib.reload(mn)
# importlib.reload(fn)
import torch
import torch.nn as nn

In [2]:
# # 这四种等价，都是一个对一个
# print(mn.layer('fc',10))
# print(mn.layer('fc',[10]))
# print(mn.layer(['fc'],10))
# print(mn.layer(['fc'],[10]))

# # 第一维度，一一对应，缺少部分重复最后一个到一样长
# print(mn.layer('fc',[10,20]))
# print(mn.layer(['fc'],[10,20]))
# print(mn.layer(['fc','act'],[10,20]))
# print(mn.layer(['fc','act'],10))
# print(mn.layer(['fc','act'],[10]))

# 在第二维度，网络代表当成一个整体，而输出代表重复
# print(mn.layer(['fc','act'],[[10,20],30]))
# print(mn.layer([['fc','bn'],'act'],[10,20]))
print(mn.layer([['fc','bn'],'act'],[[10,20],30]))

# 再复杂的网络采用*或者+连接

[[10, 20], 30] [['fc', 'bn'], 'act']
[10, 20] [['fc', 'bn'], ['fc', 'bn']]
[10, 10] ['fc', 'bn']
[20, 20] ['fc', 'bn']
SEQ>FLOW(
  (0:mix): SEQ>FLOW(
    (cell-0): SEQ>FLOW(
      (0:fc): @dup:Linear(in_features=0, out_features=10, bias=True) *id:22574603525024
      (1:bn): @dup:BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) *id:22574603528048
    )
    (cell-1): SEQ>FLOW(
      (0:fc): @dup:Linear(in_features=10, out_features=20, bias=True) *id:22574611247024
      (1:bn): @dup:BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) *id:22574611246832
    )
  )
  (1:act): @dup:GELU(approximate='none') *id:22574611245392
)


/home/myf/Skorch/.venv/lib/python3.10/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [6]:
F=mn.Mix(0,[10,[10,10],1])
F1=F*(fn(max)+min)
F1

[10, [10, 10], 1] ['dp_0.5', ['fc', 'bn', 'act', 'dp_0.5'], 'fc']
[10, 10] [['fc', 'bn', 'act', 'dp_0.5'], ['fc', 'bn', 'act', 'dp_0.5']]
[10, 10, 10, 10] ['fc', 'bn', 'act', 'dp_0.5']
[10, 10, 10, 10] ['fc', 'bn', 'act', 'dp_0.5']


SEQ>FuncModel(
  (0): SEQ>FLOW(
    (0:dp): @dup:Dropout(p=0.5, inplace=False) *id:22463246169088
    (1:mix): SEQ>FLOW(
      (cell-0): SEQ>FLOW(
        (0:fc): @dup:Linear(in_features=10, out_features=10, bias=True) *id:22463235639808
        (1:bn): @dup:BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) *id:22466472420656
        (2:act): @dup:GELU(approximate='none') *id:22463246168320
        (3:dp): @dup:Dropout(p=0.5, inplace=False) *id:22463246169952
      )
      (cell-1): SEQ>FLOW(
        (0:fc): @dup:Linear(in_features=10, out_features=10, bias=True) *id:22463246168560
        (1:bn): @dup:BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) *id:22463246167792
        (2:act): @dup:GELU(approximate='none') *id:22463246170288
        (3:dp): @dup:Dropout(p=0.5, inplace=False) *id:22466472420992
      )
    )
    (2:fc): @dup:Linear(in_features=10, out_features=1, bias=True) *id:22463246159008
  )
  (1): LIC>FuncModel(

In [7]:
F.p(torch.randn(2,10))

@ SEQ>Mix( >>
    tensor([[ 0.3124,  0.1198, -1.0895,  1.7064,  1.1035,  1.0846,  1.2973, -0.1490,
         -0.6608,  1.1761],
        [ 1.9831,  0.8707, -0.8172,  0.4615, -1.1719,  1.4913, -2.2713,  0.2525,
          0.1307,  1.6151]])
@ SEQ>FLOW( >>
    tensor([[ 0.3124,  0.1198, -1.0895,  1.7064,  1.1035,  1.0846,  1.2973, -0.1490,
         -0.6608,  1.1761],
        [ 1.9831,  0.8707, -0.8172,  0.4615, -1.1719,  1.4913, -2.2713,  0.2525,
          0.1307,  1.6151]])
  0:dp : @dup:Dropout(p=0.5, inplace=False) *id:22463246169088
  $ (tensor([[ 0.3124,  0.1198, -1.0895,  1.7064,  1.1035,  1.0846,  1.2973, -0.1490,
         -0.6608,  1.1761],
        [ 1.9831,  0.8707, -0.8172,  0.4615, -1.1719,  1.4913, -2.2713,  0.2525,
          0.1307,  1.6151]]),) >>
  == tensor([[ 0.6248,  0.2397, -0.0000,  0.0000,  0.0000,  2.1691,  2.5947, -0.2981,
         -0.0000,  0.0000],
        [ 0.0000,  0.0000, -1.6344,  0.9229, -0.0000,  2.9827, -0.0000,  0.5050,
          0.2615,  0.0000]])
@ SEQ>FLO

tensor([[ 0.3060],
        [-0.3956]], grad_fn=<AddmmBackward0>)